In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
cd /content/drive/MyDrive/Australia/'QUT IT'/'IFN509 Data Exploration and Mining'/Ass2

/content/drive/MyDrive/Australia/QUT IT/IFN509 Data Exploration and Mining/Ass2


In [22]:
# Load dataset from google drive
import pandas as pd
import numpy as np

df = pd.read_csv('D1.csv')

In [23]:
pip install apyori 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1

In [50]:
df['Customer_ID']=df['Customer_ID'].astype(str)

In [51]:
df['Sales_ID']=df['Sales_ID'].astype(str)

In [52]:
transactions1 = df.groupby(['Sales_ID'])['SKU_Category'].apply(list)
print(transactions1.head(5))

Sales_ID
1             [X52]
10       [N8U, JR5]
100           [Q4N]
1000          [N8U]
10000         [B93]
Name: SKU_Category, dtype: object


# 2

In [53]:
from apyori import apriori
# type cast the transactions from pandas into normal list format and run apriori
transaction_list = list(transactions1)
results = list(apriori(transaction_list, min_support=0.005))
# print first 5 rules
print(results[:5])


[RelationRecord(items=frozenset({'01F'}), support=0.02682353668717727, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'01F'}), confidence=0.02682353668717727, lift=1.0)]), RelationRecord(items=frozenset({'0H2'}), support=0.06429918679076095, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'0H2'}), confidence=0.06429918679076095, lift=1.0)]), RelationRecord(items=frozenset({'0KX'}), support=0.006802510744874927, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'0KX'}), confidence=0.006802510744874927, lift=1.0)]), RelationRecord(items=frozenset({'0WT'}), support=0.010296527627469776, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'0WT'}), confidence=0.010296527627469776, lift=1.0)]), RelationRecord(items=frozenset({'1EO'}), support=0.013697782999907239, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'1EO'}), confidenc

In [54]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 
                                        'Confidence', 'Lift']) 

result_df = convert_apriori_results_to_pandas_df(results)


In [55]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df[:10])

    Left_side Right_side   Support  Confidence      Lift
315   N8U,LPF        OXH  0.006973    0.413761  9.919540
314       OXH    N8U,LPF  0.006973    0.167161  9.919540
308   IEV,N8U        OXH  0.006076    0.396569  9.507370
307       OXH    IEV,N8U  0.006076    0.145663  9.507370
138       9ZX        FU5  0.007220    0.380293  9.100304
139       FU5        9ZX  0.007220    0.172771  9.100304
277       FU5    OXH,LPF  0.007313    0.174991  8.720148
282   OXH,LPF        FU5  0.007313    0.364407  8.720148
312       LPF    N8U,OXH  0.006973    0.097010  8.715011
317   N8U,OXH        LPF  0.006973    0.626389  8.715011


# 3 

In [56]:
ans = result_df.loc[result_df['Left_side']=='01F']
ans.sort_values(by = 'Lift', ascending = False).head(5)

,Left_side,Right_side,Support,Confidence,Lift
249,01F,"IEV,LPF",0.006447,0.240346,7.715161
87,01F,IEV,0.012909,0.481268,7.553841
81,01F,6BZ,0.005612,0.209222,7.460249
96,01F,OXH,0.008163,0.304323,7.295851
84,01F,FU5,0.007498,0.279539,6.689284


# 4

In [57]:
transactions = df.groupby(['Sales_ID'])['SKU_Category'].apply(list)
sequences = transactions.values.tolist()
# show the first 5 sequences
print(sequences[:5])

[['X52'], ['N8U', 'JR5'], ['Q4N'], ['N8U'], ['B93']]


# 5

In [34]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    return output_dict
    # run SPMF with supplied parameters
def second_step(output_dict):
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

In [35]:
second_step(get_association_rules(sequences, 0.01, 0.01))

,Left_rule,Right_rule,Support,Confidence
0,[Q4N],[N8U],0.001500,0.020058
1,[N8U],[Q4N],0.001500,0.008888
2,[B93],[N8U],0.000557,0.019879
3,[N8U],[B93],0.000557,0.003299
4,[FU5],[N8U],0.001886,0.023926
...,...,...,...,...
383,[LGI],[XAE],0.000387,0.013383
384,[0KX],[LGI],0.000881,0.215094
385,[LGI],[0KX],0.000881,0.030514
386,[B7G],[LGI],0.000510,0.235714


In [36]:
outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')

In [ ]:
transaction_list = []
for cust_id in df['Customer_ID'].unique():
  temp=df[df['Customer_ID'] == cust_id]
  transactions1 = temp.groupby(['Sales_ID'])['SKU_Category'].apply(list)
  transaction_list.append(list(transactions1))

transaction_list

In [39]:
# This function does not work because it can't access the SPMF

def get_association_rules_2nd(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            output_dict[str(item_id)] = itemset   ## Extra
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 
                     'seq_rule_input.txt', 'seq_rule_output.txt', 
                     supp_param, conf_param], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

# Support(Itemset) = Frequency of Itemsets/Total transaction
# Confident(A => B) = Support([A,B]) / Support([A])
# Higher confindent means that the A result in B

In [42]:
get_association_rules_2nd(transaction_list, 0.01, 0.01)

,Left_rule,Right_rule,Support,Confidence
0,"[[N8U, TEU, P42, QCG]]","[[N8U, X52, X52]]",0.004287,0.020058
1,"[[N8U, X52, X52]]","[[N8U, TEU, P42, QCG]]",0.004287,0.008888
2,"[[N8U, TEU, P42, QCG]]","[[N8U, X52, X52]]",0.001591,0.019879
3,"[[N8U, X52, X52]]","[[N8U, TEU, P42, QCG]]",0.001591,0.003299
4,"[[TVL, R6E]]","[[N8U, X52, X52]]",0.005392,0.023926
...,...,...,...,...
383,"[[1VL, N8U, 01F, IEV, LPF]]","[[TZB, RU6]]",0.001105,0.013383
384,"[[LSD, 1EO, BEQ, P42, H15, DJI]]","[[1VL, N8U, 01F, IEV, LPF]]",0.002519,0.215094
385,"[[1VL, N8U, 01F, IEV, LPF]]","[[LSD, 1EO, BEQ, P42, H15, DJI]]",0.002519,0.030514
386,"[[1VL, 61C]]","[[1VL, N8U, 01F, IEV, LPF]]",0.001459,0.235714
